In [1]:
# http://karpathy.github.io/neuralnets/
#
# Via Hacker news: https://news.ycombinator.com/item?id=18840747

# This eventually turned into Andrej Karpathy's class at 
# Stanford, CS231n. The class notes are here: 
# http://cs231n.github.io/
#
# A lot of the compute graph and backprop details in the hacker's 
# guide is covered in this specific class, starting near this time: 
# https://www.youtube.com/watch?v=i94OvYb6noo&t=207s

# Note: I converted @karpathy's Javascript in the guide to Python

# Chapter 1: Real-valued Circuits

## Base Case: Single Gate in the Circuit

Lets first consider a single, simple circuit with one gate. Here’s an example:

![Single Gate](./single.png)

This circuit takes two inputs, `x` and `y`, and computes `x * y` with the `*` gate (there can be `*` gates, `+` gates, `max` gates, etc).

In [29]:
"""
Let's see the code for this gate in action:

f(x,y)=xy
"""
def forward_multiply_gate(x, y):
    return x * y
output = forward_multiply_gate(-2, 3)
print("Output:", output)

Output: -6


## The Goal

The problem we are interested in studying looks as follows:

We provide a given circuit some specific input values (e.g. `x = -2`, `y = 3`). The circuit computes an output value (e.g. `-6`).

The core question then becomes: **How should one tweak the input(s) slightly to increase the output?**

In this case, in what direction should we change `x,y` to get a number larger than `-6`?

Note that, for example, `x = -1.99` and `y = 2.99` gives `x * y = -5.95`, which is higher than `-6.0`. Don’t get confused by this: `-5.95` is better (higher) than `-6.0`. It’s an improvement of `0.05`, even though the magnitude of `-5.95` (the distance from zero) happens to be lower.

In [28]:
## Let's see how adjusting X and Y affect the output
output = forward_multiply_gate(-1.99, 2.99)
print("Output:", output)

Output: -5.950100000000001


## Strategy 1: Random Local Search

A simple gate like this is trivial, you can easily adjust inputs to get a desired value.

In [4]:
import random
x = -2
y = 3

# Try changing x,y randomly small amounds and keep track of what works
tweak_amount = 0.01
best_out = -float('inf') # infinity
best_x = x
best_y = y

for k in range(0,100):
    x_try = x + tweak_amount * (random.randint(0,10) * 2 - 1)
    y_try = y + tweak_amount * (random.randint(0,10) * 2 - 1)
    out = forward_multiply_gate(x_try, y_try)
    
    if out > best_out:
        best_out = out
        best_x = x_try
        best_y = y_try

# We should have something < -6        
print("Best X: {}, Best Y: {}, Total: {})".format(best_x, best_y, best_x * best_y))      

Best X: -1.81, Best Y: 2.99, Total: -5.4119)


So, we’re done, right? Not quite: This is a perfectly fine strategy 
for tiny problems with a few gates if you can afford the compute 
time, but it won’t do if we want to eventually consider huge 
circuits with millions of inputs. 

It turns out that we can do much better.

## Strategy 2: Numerical Gradient

In [27]:
"""
Here's how to imagine this. Imagine pulling on the output value to make it 
larger. It might exert a force on X that makes the output higher, than -6 e.g.
"""
output = forward_multiply_gate(x + 1, y) # Let's "pull" on x
print("Output:", output)

Output: -5


Think of the "pulling" of an input value in a positive or negative direction. The forces affecting the values are known as the **derivative** of the output value with respect to the inputs (`x` and `y`). 

**The derivative can be thought of as a force on each input as we pull on the output to become higher.**

In [51]:
"""
We've just learned about the "derivative" of the output value with respect to 
its inputs (x and y).
 
It's a very simple procedure. Instead of pulling on the circuit’s output, we’ll 
iterate over every input one by one, increase it very slightly and look at what 
happens to the output value. The amount the output changes in response is the 
derivative.

Here's the formula for the derivative with respect to x:

∂f(x, y)     f(x + h, y) - f(x, y)
--------  =  -------------------
∂x                     h


A "derivative" is with respect to a single input. The gradient is a collection 
of ALL the derivatives. (It's represented as a concatendated list, a vector--not shown.)

"""
x = -2
y = 3
h = 0.0001
output = forward_multiply_gate(x, y)

# Compute the derivative with respect to x
# (...What happens when we turn the knob x to x + h)
xph = x + h
print("X plus H:", xph) # -1.9999
output2 = forward_multiply_gate(xph, y)
print("Output with tweaked X:", output2) # -5.9997
derivative_x = (output2 - output) / h
print("x':", derivative_x)
print()

# Compute the derivative with respect to y
# (...What happens when we turn the knob y to y + h)
yph = y + h
print("Y plus H:", yph) # 3.0001
output3 = forward_multiply_gate(x, yph)
print("Output with tweaked Y:", output3) # -6.0002
derivative_y = (output3 - output) / h
print("y':", derivative_y)
print()

before_tweak = forward_multiply_gate(x, y)
print("Before gradient:", before_tweak)
output = forward_multiply_gate(derivative_x, derivative_y);
print("Output after gradient:", output)

X plus H: -1.9999
Output with tweaked X: -5.9997
x': 3.00000000000189

Y plus H: 3.0001
Output with tweaked Y: -6.0002
y': -2.0000000000042206

Before gradient: -6
Output after gradient: -6.000000000016442


Now if we allow the inputs respond to the tub gy following the gradient a tiny amount (i.e. we just add the derivative on top of every input), we can see the value increases, as expected:

In [56]:
step_size = 0.01
x = -2
y = 3

output = forward_multiply_gate(x, y) # Before, -6
print("Output:",output)

x = x + (step_size * derivative_x) # x becomes -1.97
print("x:", x)

y = y + (step_size * derivative_y) # y becomes 2.98
print("y:", y)

output_new = forward_multiply_gate(x, y) # -5.87, which achieves the goal of being greater than the original -6
print("Final output:", output_new)

Output: -6
x: -1.969999999999981
y: 2.979999999999958
Final output: -5.87059999999986


## Strategy 3: Analytic Gradient

In the previous section we evaluated the gradient by probing the 
circuit’s output value, independently for every input. This procedure 
gives you what we call a numerical gradient. This approach, however, 
is still expensive because we need to compute the circuit’s output as 
we tweak every input value independently a small amount. So the 
complexity of evaluating the gradient is linear in number of inputs. 
But in practice we will have hundreds, thousands or (for neural networks) 
even tens to hundreds of millions of inputs, and the circuits aren’t 
just one multiply gate but huge expressions that can be expensive to 
compute. We need something better.

Luckily, there is an easier and much faster way to compute the gradient: 
we can use calculus to derive a direct expression for it that will be as
simple to evaluate as the circuit’s output value. We call this an analytic
gradient and there will be no need for tweaking anything.

**The analytic derivative requires no tweaking of the inputs. It can be derived using mathematics (calculus).**


In [57]:
"""
For our function, f(x, y) = (x * y), the derivative of x is y, 
and the derivative of y is x:
"""

x = -2
y = 3
output = forward_multiply_gate(x, y) # Before: -6

x_gradient = y
y_gradient = x

step_size = 0.01
x += step_size * x_gradient # -1.97
y += step_size * y_gradient # 2.98

output_new = forward_multiply_gate(x,y)
print(output_new) # -5.87, which achieves the goal of being greater than the original -6

-5.8706


## Recursive Case: Circuits with Multiple Gates

“The analytic gradient was trivial to derive for your super-simple expression. This is useless. What do I do when the expressions are much larger? Don’t the equations get huge and complex very fast?”. Good question. Yes the expressions get much more complex. No, this doesn’t make it much harder. As we will see, every gate will be hanging out by itself, completely unaware of any details of the huge and complex circuit that it could be part of. It will only worry about its inputs and it will compute its local derivatives as seen in the previous section, except now there will be a single extra multiplication it will have to do.

![Recursive](./recursive_case.png)

The expression we are computing now is f(x,y,z) = (x+y)z. 

In [58]:
# The expression we are now computing is f(x, y, z) = (x + y)z

def forward_multiply_gate(a, b): # Redfining inputs as a, b to indicate they are local
    return a * b                 # variables and not to be confused with x, y, z later

def forward_add_gate(a, b):
    return a + b

def forward_circuit(x, y, z):
    q = forward_add_gate(x, y)
    f = forward_multiply_gate(q, z)
    return f

x = -2
y = 5
z = -4

f = forward_circuit(x, y, z) 
print("f: ", output) # output is -12

f:  -6
